# gssnng
Gene Set Scoring on the Nearest Neighbor Graph (gssnng) for Single Cell RNA-seq (scRNA-seq)
Works with AnnData objects stored as h5ad files. Takes values from adata.X.

https://github.com/Gibbsdavidl/gssnng

Notebook author: David L Gibbs (david.gibbs@isbscience.org)

In [ ]:
# first let's install the package from github
# and clone the repo for the example data
!pip install git+https://github.com/Gibbsdavidl/gssnng
!git clone https://github.com/Gibbsdavidl/gssnng

In [ ]:
from gssnng import score_cells
import scanpy as sc
import matplotlib
%matplotlib inline

In [ ]:
# read in the 10x genomics example data set, same as used in the scanpy tuts
q = sc.read_h5ad('gssnng/gssnng/test/data/pbmc3k_processed.h5ad')

In [ ]:
# recompute the nearest neighbor graph to give plenty of neighbors to each cell
sc.pp.neighbors(q, n_neighbors=32)

In [ ]:
# then we'll produce a gene set score, for each cell, for each gene set in the .gmt file
score_cells.with_gene_sets(
        adata=q,
        gene_set_file="gssnng/gssnng/test/data/cibersort_lm22.gmt",
        groupby="louvain",
        recompute_neighbors=0,
        score_method="robust_std",
        method_params=dict(),
        samp_neighbors=29,
        noise_trials=0,
        ranked=True
        cores=8
    )

In [ ]:
# each gene set is saved as a column in the AnnData.obs data.frame
q.obs

In [ ]:
# let's visualize the CD8 T cell scores
sc.pl.umap(q, color=['louvain','T.cells.CD8'], wspace=0.35)